In [1]:
import json
from fastapi import FastAPI
import gensim

import scipy

import sys
sys.path.append('./models')

from fastapi.middleware.cors import CORSMiddleware

import pickle 
import os
import glob

In [2]:
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

In [3]:
app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [29]:

def reated_documents(user_input):
    words = user_input.split()
    rel_docs = set()
    for word in words:
        myquery = {"word": word}
        results  = mycollection.find(myquery)
        for result in results:
            docs = result['doc_ids']
            rel_docs.update(docs)
            
    return list(rel_docs)

def url_documents(rel_docs):
    documents  = []
    urls = []
    for idx in rel_docs:
        query = f"SELECT * FROM sites where id = {idx};"
        cursor.execute(query)
        results = cursor.fetchall()
        for row in results:
            doc_id = row[0]
            url = row[1]
            urls.append(url)
            text = row[2]
            documents.append(text)
    return (documents,urls)

def BM25_json(user_query):
    out_data = {}
    rew_score_list = []
    
    rel_docs = reated_documents(user_query)
    documents , urls = url_documents(rel_docs)
    
    tokenized_corpus = [doc.split(" ") for doc in documents]
    bm25 = BM25Okapi(tokenized_corpus)
    
    tokenized_query = user_query.split()
    doc_scores_raw = bm25.get_scores(tokenized_query)
    doc_scores_indxed = doc_scores_raw.argsort()[::-1]
    doc_scores_indxed = doc_scores_indxed[:10]
    sorted_rawscores = doc_scores_raw[doc_scores_indxed[:10]]
    
    query = f"SELECT max(number)+1 FROM sessions;"
    cursor.execute(query)
    session_id = cursor.fetchall()[0][0]
    session_id =int(session_id)
    
    for idx,idx_score in enumerate(zip(doc_scores_indxed,sorted_rawscores),start=1):
        doc_idx, raw_score = idx_score[0], idx_score[1]
        rew_score_list.append(raw_score)
        doc_id = rel_docs[doc_idx]
        url = urls[doc_idx]
        text = documents[doc_idx][:600]
        out_data.update({idx:{'rank':idx,'doc_id':doc_id,'score':raw_score,'url':url,'text':text,'session_id':session_id }})  
    
    return out_data,rew_score_list

In [3]:
from models.bm25_base import BM25_json

In [4]:
user_input = 'London Rishi'

In [5]:
out_data, raw_scores = BM25_json(user_input)

In [6]:
json_data = json.dumps(out_data)

In [7]:
json_data

'{"1": {"rank": 1, "doc_id": 1610, "score": 5.756529688123202, "url": "https://www.bbc.com/news/uk-england-london-64803166", "text": "Tube strike: RMT announces London Underground walk out More London Underground staff are to strike on 15 March, Budget Day, a transport workers\' union has announced. The RMT has announced the walkout, which is in a dispute over pensions, job losses and contractual agreements. Members will walk out on the same day that Aslef members are striking on the Tube network. A spokesperson for Transport for London (TfL) said: \\"We have not proposed changes to anyone\'s pensions.\\" But the RMT\'s general secretary Mick Lynch said: \\"Our members will never accept job losses, attacks on their pensions or change", "session_id": 1}, "2": {"rank": 2, "doc_id": 906, "score": 5.699376436595253, "url": "https://www.bbc.com/news/uk-northern-ireland-64795902", "text": "Brexit: What is the Stormont brake? This video can not be played Watch: How would the Stormont brake wo

In [24]:
doc_id

1610

In [25]:
query = f"SELECT * FROM sites where id = {doc_id};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(1610, 'https://www.bbc.com/news/uk-england-london-64803166', 'Tube strike: RMT announces London Underground walk out More London Underground staff are to strike on 15 March, Budget Day, a transport workers\' union has announced. The RMT has announced the walkout, which is in a dispute over pensions, job losses and contractual agreements. Members will walk out on the same day that Aslef members are striking on the Tube network. A spokesperson for Transport for London (TfL) said: "We have not proposed changes to anyone\'s pensions." But the RMT\'s general secretary Mick Lynch said: "Our members will never accept job losses, attacks on their pensions or changes to working conditions in order to pay for a funding cut which is the government\'s political decision. "Tube workers provide an essential service to the capital, making sure the city can keep moving and work long hours in demanding roles. "In return they deserve decent pensions, job security and good working conditions and the RMT

## TESTING WORD2VEC

As mentioned in the DESM paper, we compute the centroid of documents and compute the cosine similarity of the query words with document centroid.

In [4]:
from models.desm import DESM_json
from models.bm25_base import BM25_json

import json

In [5]:
out_data, raw_scores = DESM_json('Rishi London')
out_data_bm25, raw_scores_bm25 = BM25_json('Rishi London')

{1: {'rank': 1, 'doc_id': 357, 'score': 0.5438157918094071, 'url': 'https://www.bbc.com/news/uk-politics-64780244', 'text': 'NI Protocol: Final talks on Brexit deal due - No 10 Rishi Sunak and the European Commission president are due to hold "final talks" on a new post-Brexit deal for Northern Ireland on Monday, Downing Street says. The prime minister and Ursula von der Leyen will meet in Berkshire in the afternoon to discuss "complex challenges" around the protocol. The UK wants to change the current agreement - which sees some goods checked when entering Northern Ireland from the rest of Great Britain. A new deal has been expected for days. Deputy Prime Minister Dominic Raab earlier said the UK and EU were "on th', 'session_id': 1}, 2: {'rank': 2, 'doc_id': 235, 'score': 0.540018284813073, 'url': 'https://www.bbc.com/news/uk-politics-64773214', 'text': 'Northern Ireland Protocol: Sunak \'giving it everything\' to reach Brexit deal Rishi Sunak says he is "giving it everything" this w

In [6]:
json.dumps(out_data)

'{"1": {"rank": 1, "doc_id": 357, "score": 0.5438157918094071, "url": "https://www.bbc.com/news/uk-politics-64780244", "text": "NI Protocol: Final talks on Brexit deal due - No 10 Rishi Sunak and the European Commission president are due to hold \\"final talks\\" on a new post-Brexit deal for Northern Ireland on Monday, Downing Street says. The prime minister and Ursula von der Leyen will meet in Berkshire in the afternoon to discuss \\"complex challenges\\" around the protocol. The UK wants to change the current agreement - which sees some goods checked when entering Northern Ireland from the rest of Great Britain. A new deal has been expected for days. Deputy Prime Minister Dominic Raab earlier said the UK and EU were \\"on th", "session_id": 1}, "2": {"rank": 2, "doc_id": 235, "score": 0.540018284813073, "url": "https://www.bbc.com/news/uk-politics-64773214", "text": "Northern Ireland Protocol: Sunak \'giving it everything\' to reach Brexit deal Rishi Sunak says he is \\"giving it e

In [10]:
json.dumps(out_data_bm25)

'{"1": {"rank": 1, "doc_id": 1610, "score": 5.756529688123202, "url": "https://www.bbc.com/news/uk-england-london-64803166", "text": "Tube strike: RMT announces London Underground walk out More London Underground staff are to strike on 15 March, Budget Day, a transport workers\' union has announced. The RMT has announced the walkout, which is in a dispute over pensions, job losses and contractual agreements. Members will walk out on the same day that Aslef members are striking on the Tube network. A spokesperson for Transport for London (TfL) said: \\"We have not proposed changes to anyone\'s pensions.\\" But the RMT\'s general secretary Mick Lynch said: \\"Our members will never accept job losses, attacks on their pensions or change", "session_id": 1}, "2": {"rank": 2, "doc_id": 906, "score": 5.699376436595253, "url": "https://www.bbc.com/news/uk-northern-ireland-64795902", "text": "Brexit: What is the Stormont brake? This video can not be played Watch: How would the Stormont brake wo

In [4]:
import json